[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joshhu/llmdistill/blob/main/llm_distillation_demo.ipynb)

# LLM Knowledge Distillation 示範

這個 notebook 示範如何使用知識蒸餾（Knowledge Distillation）來訓練語言模型。

## 流程說明
1. **老師模型（Teacher）**: GPT-2 (117M 參數) - 用來生成訓練資料
2. **學生模型（Student）**: 較小的 GPT-2 配置 (約 40M 參數) - 從老師生成的資料學習
3. **評估指標**: 使用 Perplexity 來衡量模型效果

## 硬體需求
- GPU: 16GB VRAM (Colab Free Tier T4 可執行)
- 訓練時間: 約 15-20 分鐘

In [ ]:
# 安裝必要套件
!pip install -q transformers datasets torch accelerate sentencepiece

# 安裝中文字體支援（在 Colab 環境）
!apt-get install -qq fonts-noto-cjk
import matplotlib.font_manager as fm
fm._rebuild()

In [ ]:
import torch
import torch.nn as nn
from transformers import (
    GPT2LMHeadModel, 
    GPT2Tokenizer, 
    GPT2Config,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 設定 matplotlib 中文字體支援
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial Unicode MS', 'SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False  # 解決負號顯示問題

# 檢查 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用裝置: {device}")
if torch.cuda.is_available():
    print(f"GPU 名稱: {torch.cuda.get_device_name(0)}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 步驟 1: 載入老師模型

In [ ]:
# 載入老師模型 (GPT-2)
print("載入老師模型 (GPT-2)...")
teacher_model_name = "gpt2"
teacher_model = GPT2LMHeadModel.from_pretrained(teacher_model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(teacher_model_name)
tokenizer.pad_token = tokenizer.eos_token

# 統計老師模型參數量
teacher_params = sum(p.numel() for p in teacher_model.parameters())
print(f"老師模型參數量: {teacher_params:,} ({teacher_params/1e6:.1f}M)")

## 步驟 2: 使用老師模型生成訓練資料

In [ ]:
def generate_training_data(model, tokenizer, prompts, num_samples=100, max_length=128):
    model.eval()
    generated_texts = []
    
    print(f"\n生成 {num_samples} 個訓練樣本...")
    
    with torch.no_grad():
        for i in tqdm(range(num_samples)):
            prompt = prompts[i % len(prompts)]
            
            # 使用 tokenizer() 而非 encode() 以獲得 attention_mask
            inputs = tokenizer(prompt, return_tensors='pt').to(device)
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']
            
            output = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.9,
                top_k=50,
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
            
            text = tokenizer.decode(output[0], skip_special_tokens=True)
            generated_texts.append(text)
    
    return generated_texts

# 定義多樣化的提示詞
prompts = [
    "Once upon a time",
    "The future of technology",
    "In a world where",
    "Scientists have discovered",
    "The story begins with",
    "Artificial intelligence is",
    "Climate change has",
    "In the year 2050",
    "The most important thing",
    "Machine learning can"
]

# 生成訓練資料
num_training_samples = 500
generated_texts = generate_training_data(
    teacher_model, 
    tokenizer, 
    prompts, 
    num_samples=num_training_samples,
    max_length=100
)

print(f"\n生成完成! 共 {len(generated_texts)} 個樣本")
print("\n範例生成文本:")
for i in range(3):
    print(f"\n[{i+1}] {generated_texts[i][:200]}...")

## 步驟 3: 建立學生模型

In [ ]:
# 建立較小的學生模型配置
print("\n建立學生模型...")
student_config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=1024,
    n_embd=512,
    n_layer=6,
    n_head=8,
    n_inner=2048,
)

student_model = GPT2LMHeadModel(student_config).to(device)

student_params = sum(p.numel() for p in student_model.parameters())
print(f"學生模型參數量: {student_params:,} ({student_params/1e6:.1f}M)")
print(f"參數壓縮比: {teacher_params/student_params:.2f}x")

## 步驟 4: 準備訓練資料集

In [ ]:
def prepare_dataset(texts, tokenizer, max_length=128):
    encodings = tokenizer(
        texts, 
        truncation=True, 
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )
    
    # 確保返回正確的張量格式
    dataset_dict = {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask']
    }
    
    # 設置格式為 torch
    dataset = Dataset.from_dict(dataset_dict)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    
    return dataset

print("\n準備訓練資料集...")
split_idx = int(len(generated_texts) * 0.9)
train_texts = generated_texts[:split_idx]
val_texts = generated_texts[split_idx:]

train_dataset = prepare_dataset(train_texts, tokenizer, max_length=100)
val_dataset = prepare_dataset(val_texts, tokenizer, max_length=100)

print(f"訓練集大小: {len(train_dataset)}")
print(f"驗證集大小: {len(val_dataset)}")

## 步驟 5: 評估函數 - 計算 Perplexity

In [ ]:
def calculate_perplexity(model, dataset, tokenizer, batch_size=8):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    from torch.utils.data import DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="計算 Perplexity"):
            # 確保數據是 tensor 格式並移到正確的設備
            if isinstance(batch['input_ids'], list):
                input_ids = torch.tensor(batch['input_ids']).to(device)
                attention_mask = torch.tensor(batch['attention_mask']).to(device)
            else:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=input_ids
            )
            
            loss = outputs.loss
            total_loss += loss.item() * input_ids.size(0)
            total_tokens += input_ids.size(0)
    
    avg_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(avg_loss))
    
    return perplexity.item()

print("\n評估訓練前的學生模型...")
initial_perplexity = calculate_perplexity(student_model, val_dataset, tokenizer)
print(f"訓練前 Perplexity: {initial_perplexity:.2f}")

## 步驟 6: 訓練學生模型

In [ ]:
training_args = TrainingArguments(
    output_dir="./student_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

print("\n開始訓練學生模型...")
print("=" * 50)
trainer.train()

## 步驟 7: 評估訓練後的學生模型

In [ ]:
print("\n評估訓練後的學生模型...")
final_perplexity = calculate_perplexity(student_model, val_dataset, tokenizer)
print(f"訓練後 Perplexity: {final_perplexity:.2f}")

improvement = (initial_perplexity - final_perplexity) / initial_perplexity * 100
print(f"\n改善率: {improvement:.2f}%")

print("\n評估老師模型 (作為對照)...")
teacher_perplexity = calculate_perplexity(teacher_model, val_dataset, tokenizer)
print(f"老師模型 Perplexity: {teacher_perplexity:.2f}")

## 步驟 7.5: 計算 KL Divergence（分布相似度）

KL Divergence 衡量學生模型和老師模型輸出分布的相似度，是評估知識蒸餾效果的重要指標。

In [ ]:
def calculate_kl_divergence(teacher, student, dataset, tokenizer, batch_size=8, num_samples=50):
    """
    計算學生模型和老師模型輸出分布的 KL Divergence
    
    Args:
        teacher: 老師模型
        student: 學生模型
        dataset: 評估資料集
        tokenizer: 分詞器
        batch_size: 批次大小
        num_samples: 計算的樣本數量（為了效率）
    
    Returns:
        平均 KL divergence 值
    """
    import torch.nn.functional as F
    
    teacher.eval()
    student.eval()
    
    total_kl = 0
    count = 0
    
    from torch.utils.data import DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    print(f"\n計算 KL Divergence（使用 {num_samples} 個樣本）...")
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="計算 KL Divergence"):
            if count >= num_samples:
                break
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # 獲取老師和學生的 logits
            teacher_outputs = teacher(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            student_outputs = student(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            teacher_logits = teacher_outputs.logits
            student_logits = student_outputs.logits
            
            # 計算 KL divergence
            # KL(P||Q) = sum(P * log(P/Q))
            kl_div = F.kl_div(
                F.log_softmax(student_logits, dim=-1),
                F.softmax(teacher_logits, dim=-1),
                reduction='batchmean'
            )
            
            total_kl += kl_div.item()
            count += 1
    
    avg_kl = total_kl / count
    return avg_kl

# 計算 KL Divergence
kl_divergence = calculate_kl_divergence(
    teacher_model, 
    student_model, 
    val_dataset, 
    tokenizer,
    num_samples=50
)

print(f"\n學生模型與老師模型的 KL Divergence: {kl_divergence:.4f}")
print(f"解釋: KL Divergence 越小，表示學生模型的輸出分布越接近老師模型")

## 步驟 8: 視覺化結果

In [ ]:
models = ['學生模型\n(訓練前)', '學生模型\n(訓練後)', '老師模型']
perplexities = [initial_perplexity, final_perplexity, teacher_perplexity]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

plt.figure(figsize=(10, 6))
bars = plt.bar(models, perplexities, color=colors, alpha=0.8, edgecolor='black')
plt.ylabel('Perplexity (越低越好)', fontsize=12, fontweight='bold')
plt.title('Knowledge Distillation 效果比較', fontsize=14, fontweight='bold', pad=20)
plt.grid(axis='y', alpha=0.3, linestyle='--')

for bar, ppl in zip(bars, perplexities):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{ppl:.2f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('distillation_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n圖表已儲存為 'distillation_results.png'")

## 步驟 9: 生成文本比較

In [ ]:
def compare_generation(teacher, student, tokenizer, prompt, max_length=100):
    teacher.eval()
    student.eval()
    
    # 使用 tokenizer() 而非 encode() 以獲得 attention_mask
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    with torch.no_grad():
        teacher_output = teacher.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        teacher_text = tokenizer.decode(teacher_output[0], skip_special_tokens=True)
        
        student_output = student.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        student_text = tokenizer.decode(student_output[0], skip_special_tokens=True)
    
    return teacher_text, student_text

test_prompts = [
    "The future of artificial intelligence",
    "In a distant galaxy",
    "Climate change is"
]

print("\n" + "="*60)
print("文本生成比較")
print("="*60)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n【測試 {i}】提示詞: \"{prompt}\"")
    print("-" * 60)
    
    teacher_text, student_text = compare_generation(
        teacher_model, student_model, tokenizer, prompt, max_length=80
    )
    
    print(f"\n🎓 老師模型:\n{teacher_text}")
    print(f"\n🎒 學生模型:\n{student_text}")
    print("-" * 60)

## 總結報告

## 步驟 10: 生成品質評估

評估生成文本的多樣性和老師/學生模型輸出的相似度。

In [ ]:
def calculate_diversity(texts):
    """
    計算文本集合的詞彙多樣性
    
    Args:
        texts: 文本列表
    
    Returns:
        unique_token_ratio: 唯一 token 比例
        avg_length: 平均文本長度
    """
    all_tokens = []
    for text in texts:
        tokens = text.split()
        all_tokens.extend(tokens)
    
    unique_tokens = len(set(all_tokens))
    total_tokens = len(all_tokens)
    unique_token_ratio = unique_tokens / total_tokens if total_tokens > 0 else 0
    avg_length = total_tokens / len(texts) if len(texts) > 0 else 0
    
    return unique_token_ratio, avg_length

def calculate_text_similarity(texts1, texts2):
    """
    計算兩組文本的相似度（使用 Jaccard similarity）
    
    Args:
        texts1: 第一組文本列表
        texts2: 第二組文本列表
    
    Returns:
        平均 Jaccard 相似度
    """
    similarities = []
    
    for t1, t2 in zip(texts1, texts2):
        tokens1 = set(t1.split())
        tokens2 = set(t2.split())
        
        intersection = len(tokens1 & tokens2)
        union = len(tokens1 | tokens2)
        
        similarity = intersection / union if union > 0 else 0
        similarities.append(similarity)
    
    return sum(similarities) / len(similarities) if similarities else 0

# 生成測試樣本用於品質評估
print("\\n生成測試樣本用於品質評估...")
test_generation_prompts = [
    "Once upon a time",
    "The future of technology",
    "In a world where",
    "Scientists have discovered",
    "Artificial intelligence is"
]

teacher_generated = []
student_generated = []

for prompt in test_generation_prompts:
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        # 老師模型生成
        teacher_output = teacher_model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=100,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            num_return_sequences=3,  # 每個提示生成 3 個樣本
            pad_token_id=tokenizer.eos_token_id
        )
        
        for output in teacher_output:
            text = tokenizer.decode(output, skip_special_tokens=True)
            teacher_generated.append(text)
        
        # 學生模型生成
        student_output = student_model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=100,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            num_return_sequences=3,
            pad_token_id=tokenizer.eos_token_id
        )
        
        for output in student_output:
            text = tokenizer.decode(output, skip_special_tokens=True)
            student_generated.append(text)

# 計算多樣性
teacher_diversity, teacher_avg_len = calculate_diversity(teacher_generated)
student_diversity, student_avg_len = calculate_diversity(student_generated)

print(f"\\n📊 生成文本多樣性:")
print(f"  老師模型:")
print(f"    • 唯一 token 比例: {teacher_diversity:.4f}")
print(f"    • 平均文本長度: {teacher_avg_len:.1f} tokens")
print(f"  學生模型:")
print(f"    • 唯一 token 比例: {student_diversity:.4f}")
print(f"    • 平均文本長度: {student_avg_len:.1f} tokens")

# 計算老師和學生輸出的相似度
similarity = calculate_text_similarity(teacher_generated, student_generated)
print(f"\\n🔗 老師與學生輸出相似度:")
print(f"  • Jaccard 相似度: {similarity:.4f}")
print(f"  • 解釋: 相似度越高，表示學生模型越能模仿老師模型的生成風格")

## 步驟 11: 視覺化輸出分布比較

比較老師和學生模型在相同輸入下的 token probability 分布。

In [ ]:
import torch.nn.functional as F

# 選擇一個測試句子來比較分布
test_sentence = "The future of artificial intelligence is"
inputs = tokenizer(test_sentence, return_tensors='pt').to(device)

# 獲取老師和學生的輸出分布
with torch.no_grad():
    teacher_outputs = teacher_model(**inputs)
    student_outputs = student_model(**inputs)
    
    teacher_probs = F.softmax(teacher_outputs.logits[0, -1, :], dim=-1).cpu().numpy()
    student_probs = F.softmax(student_outputs.logits[0, -1, :], dim=-1).cpu().numpy()

# 獲取 top-20 最可能的 tokens
top_k = 20
teacher_top_indices = teacher_probs.argsort()[-top_k:][::-1]
student_top_indices = student_probs.argsort()[-top_k:][::-1]

# 合併兩組 top tokens
all_top_indices = list(set(teacher_top_indices) | set(student_top_indices))
all_top_indices = sorted(all_top_indices, 
                         key=lambda x: teacher_probs[x] + student_probs[x], 
                         reverse=True)[:top_k]

# 準備繪圖資料
tokens = [tokenizer.decode([idx]) for idx in all_top_indices]
teacher_top_probs = [teacher_probs[idx] for idx in all_top_indices]
student_top_probs = [student_probs[idx] for idx in all_top_indices]

# 繪製比較圖
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 左圖：並排比較
x = np.arange(len(tokens))
width = 0.35

bars1 = ax1.bar(x - width/2, teacher_top_probs, width, label='老師模型', 
                color='#45B7D1', alpha=0.8)
bars2 = ax1.bar(x + width/2, student_top_probs, width, label='學生模型', 
                color='#4ECDC4', alpha=0.8)

ax1.set_xlabel('Token', fontsize=11, fontweight='bold')
ax1.set_ylabel('機率', fontsize=11, fontweight='bold')
ax1.set_title(f'Top-{top_k} Token 機率分布比較\\n輸入: "{test_sentence}"', 
              fontsize=12, fontweight='bold', pad=15)
ax1.set_xticks(x)
ax1.set_xticklabels(tokens, rotation=45, ha='right')
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# 右圖：散點圖比較
ax2.scatter(teacher_top_probs, student_top_probs, alpha=0.6, s=100, color='#FF6B6B')
ax2.plot([0, max(teacher_top_probs)], [0, max(teacher_top_probs)], 
         'k--', alpha=0.3, label='完美匹配線')

ax2.set_xlabel('老師模型機率', fontsize=11, fontweight='bold')
ax2.set_ylabel('學生模型機率', fontsize=11, fontweight='bold')
ax2.set_title('機率分布相關性', fontsize=12, fontweight='bold', pad=15)
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3, linestyle='--')

# 添加相關係數
correlation = np.corrcoef(teacher_top_probs, student_top_probs)[0, 1]
ax2.text(0.05, 0.95, f'相關係數: {correlation:.4f}', 
         transform=ax2.transAxes, fontsize=11, 
         verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('distribution_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\\n📈 輸出分布分析:")
print(f"  • 相關係數: {correlation:.4f}")
print(f"  • 解釋: 相關係數越接近 1，表示學生模型和老師模型的輸出分布越相似")
print(f"\\n圖表已儲存為 'distribution_comparison.png'")

In [ ]:
print("\n" + "="*70)
print("Knowledge Distillation 實驗總結")
print("="*70)

print(f"\n📊 模型規格:")
print(f"  • 老師模型: GPT-2 ({teacher_params/1e6:.1f}M 參數)")
print(f"  • 學生模型: GPT-2-Small ({student_params/1e6:.1f}M 參數)")
print(f"  • 參數壓縮比: {teacher_params/student_params:.2f}x")

print(f"\n📚 訓練資料:")
print(f"  • 訓練樣本數: {len(train_texts)}")
print(f"  • 驗證樣本數: {len(val_texts)}")
print(f"  • 資料來源: 老師模型生成")

print(f"\n📈 評估結果:")
print(f"\n  1️⃣ Perplexity（預測準確度）:")
print(f"     • 老師模型: {teacher_perplexity:.2f}")
print(f"     • 學生模型 (訓練前): {initial_perplexity:.2f}")
print(f"     • 學生模型 (訓練後): {final_perplexity:.2f}")
print(f"     • 改善率: {improvement:.2f}%")

print(f"\n  2️⃣ KL Divergence（分布相似度）:")
print(f"     • KL Divergence: {kl_divergence:.4f}")
print(f"     • 解釋: 值越小表示學生越接近老師的輸出分布")

print(f"\n  3️⃣ 生成品質（多樣性）:")
print(f"     • 老師模型唯一 token 比例: {teacher_diversity:.4f}")
print(f"     • 學生模型唯一 token 比例: {student_diversity:.4f}")
print(f"     • 老師模型平均長度: {teacher_avg_len:.1f} tokens")
print(f"     • 學生模型平均長度: {student_avg_len:.1f} tokens")

print(f"\n  4️⃣ 輸出相似度:")
print(f"     • Jaccard 相似度: {similarity:.4f}")
print(f"     • 機率分布相關係數: {correlation:.4f}")

print(f"\n🎯 知識蒸餾效果分析:")
achievement_rate = (teacher_perplexity / final_perplexity) * 100
print(f"  • 學生模型達成率: {achievement_rate:.2f}%")
print(f"    (相對於老師模型的 perplexity 表現)")

efficiency_gain = (teacher_params / student_params) / (final_perplexity / teacher_perplexity)
print(f"  • 效率提升: {efficiency_gain:.2f}x")
print(f"    (參數減少 vs. 效能損失比)")

# 綜合評估
print(f"\n✅ 綜合結論:")
print(f"  ✓ 學生模型成功從老師模型生成的資料中學習!")
print(f"  ✓ 使用僅 {student_params/teacher_params*100:.1f}% 的參數量")
print(f"  ✓ Perplexity 達到老師模型 {achievement_rate:.1f}% 的效能")
print(f"  ✓ 輸出分布與老師模型相關係數: {correlation:.3f}")
print(f"  ✓ 成功實現模型壓縮，同時保持合理的生成品質")

print(f"\n💡 評估方法總結:")
print(f"  本實驗使用了多種評估指標來全面衡量知識蒸餾效果:")
print(f"  • Perplexity: 衡量預測準確度")
print(f"  • KL Divergence: 衡量分布相似度")
print(f"  • 多樣性分析: 衡量生成文本品質")
print(f"  • 相似度指標: 衡量老師/學生輸出一致性")
print(f"  • 視覺化比較: 直觀展示模型差異")

print("\n" + "="*70)